In [1]:
%%html
<style>
    pre.cd {
            border: 1px dashed #93A1A1;
            padding: 0.5em 0.8em;
            background: #FCF6E3;
    }
    .highlight{
        backgound: #FCF6E3 ;
    }
    .hll { background-color: #ffffcc }
    .c { color: #93a1a1; font-style: italic } /* Comment */
    .err { color: #dc322f } /* Error */
    .g { color: #657b83 } /* Generic */
    .k { color: #859900 } /* Keyword */
    .l { color: #657b83 } /* Literal */
    .n { color: #586e75 } /* Name */
    .o { color: #657b83 } /* Operator */
    .x { color: #657b83 } /* Other */
    .p { color: #657b83 } /* Punctuation */
    .cm { color: #93a1a1; font-style: italic } /* Comment.Multiline */
    .cp { color: #93a1a1; font-style: italic } /* Comment.Preproc */
    .c1 { color: #93a1a1; font-style: italic } /* Comment.Single */
    .cs { color: #93a1a1; font-style: italic } /* Comment.Special */
    .gd { color: #657b83 } /* Generic.Deleted */
    .ge { color: #657b83 } /* Generic.Emph */
    .gr { color: #657b83 } /* Generic.Error */
    .gh { color: #657b83 } /* Generic.Heading */
    .gi { color: #657b83 } /* Generic.Inserted */
    .go { color: #000000 } /* Generic.Output (customized) */
    .gp { color: #cb4b16 } /* Generic.Prompt (customized) */
    .gs { color: #657b83 } /* Generic.Strong */
    .gu { color: #657b83 } /* Generic.Subheading */
    .gt { color: #657b83 } /* Generic.Traceback */
    .kc { color: #859900 } /* Keyword.Constant */
    .kd { color: #859900 } /* Keyword.Declaration */
    .kn { color: #cb4b16 } /* Keyword.Namespace */
    .kp { color: #cb4b16 } /* Keyword.Pseudo */
    .kr { color: #859900 } /* Keyword.Reserved */
    .kt { color: #859900 } /* Keyword.Type */
    .ld { color: #657b83 } /* Literal.Date */
    .m { color: #2aa198 } /* Literal.Number */
    .s { color: #2aa198 } /* Literal.String */
    .na { color: #657b83 } /* Name.Attribute */
    .nb { color: #268bd2 } /* Name.Builtin */
    .nc { color: #268bd2 } /* Name.Class */
    .no { color: #b58900 } /* Name.Constant */
    .nd { color: #cb4b16 } /* Name.Decorator */
    .ni { color: #cb4b16 } /* Name.Entity */
    .ne { color: #cb4b16 } /* Name.Exception */
    .nf { color: #268bd2 } /* Name.Function */
    .nl { color: #657b83 } /* Name.Label */
    .nn { color: #b58900 } /* Name.Namespace */
    .nx { color: #657b83 } /* Name.Other */
    .py { color: #268bd2 } /* Name.Property */
    .nt { color: #859900 } /* Name.Tag */
    .nv { color: #cd4b16 } /* Name.Variable */
    .ow { color: #859900 } /* Operator.Word */
    .w { color: #fdf6e3 } /* Text.Whitespace */
    .mf { color: #2aa198 } /* Literal.Number.Float */
    .mh { color: #2aa198 } /* Literal.Number.Hex */
    .mi { color: #2aa198 } /* Literal.Number.Integer */
    .mo { color: #2aa198 } /* Literal.Number.Oct */
    .sb { color: #2aa198 } /* Literal.String.Backtick */
    .sc { color: #2aa198 } /* Literal.String.Char */
    .sd { color: #2aa198 } /* Literal.String.Doc */
    .s2 { color: #2aa198 } /* Literal.String.Double */
    .se { color: #cb4b16 } /* Literal.String.Escape */
    .sh { color: #2aa198 } /* Literal.String.Heredoc */
    .si { color: #cb4b16 } /* Literal.String.Interpol */
    .sx { color: #2aa198 } /* Literal.String.Other */
    .sr { color: #2aa198 } /* Literal.String.Regex */
    .s1 { color: #2aa198 } /* Literal.String.Single */
    .ss { color: #2aa198 } /* Literal.String.Symbol */
    .bp { color: #268bd2; font-weight: bold } /* Name.Builtin.Pseudo */
    .vc { color: #268bd2 } /* Name.Variable.Class */
    .vg { color: #268bd2 } /* Name.Variable.Global */
    .vi { color: #268bd2 } /* Name.Variable.Instance */
    .il { color: #2aa198 } /* Literal.Number.Integer.Long */
    div.verdict-div{
        background: #EEEEEE; 
        padding: 30px
    }
    p.verdict-title{
        text-align: center;
        font-weight: bold;
    }
</style>

<div class="verdict-div"><p class="verdict-title">Verdict</p>

A powerful technique for offering callables at the top level of your module that share state through a common object.</div>

# The Prebound Method Pattern
There are occasions on which a Python module wants to offer several routines in the module’s global namespace that will need to share state with each other at runtime.

Probably the most famous example is the Python Standard Library’s `random` module. While it does provide advanced users with the option to build their own `Random` instance, most programmers opt for the convenient slate of routines available in the module’s global namespace — like `randrange()`, `randint()`, and `choice()` — that mirror the methods of a `Random` object.

How do these top-level routines share state?

Behind the scenes these callables are, in fact, methods that have all been bound ahead of time to a single instance of `Random` that the module itself has gone ahead and constructed.

After reviewing the problem that the Prebound Methods pattern solves, we will see how the pattern looks in Python code.

## Alternatives
The most primitive approach to sharing state between a pair of module-level callables is to write a pair of functions that manipulate global data that’s stored next to them at the top level of the module.

Imagine that we want to offer a simple random number generator. It returns, in an endless loop, the numbers 1 through 255 in a fixed pseudo-random order. We also want to offer a simple `set_seed()` routine that resets the state of the generator to a known value — which is important both for tests that use random numbers, and for simulations driven by pseudo-randomness that want to offer reproducible results.

If Python only supported plain functions, we might store the shared seed as a module global that our functions would directly access and modify:

<pre class="cd"><span></span><span class="kn">from</span> <span class="nn">datetime</span> <span class="kn">import</span> <span class="n">datetime</span>

<span class="n">_seed</span> <span class="o">=</span> <span class="n">datetime</span><span class="o">.</span><span class="n">now</span><span class="p">()</span><span class="o">.</span><span class="n">microsecond</span> <span class="o">%</span> <span class="mi">255</span> <span class="o">+</span> <span class="mi">1</span>

<span class="k">def</span> <span class="nf">set_seed</span><span class="p">(</span><span class="n">value</span><span class="p">):</span>
    <span class="k">global</span> <span class="n">_seed</span>
    <span class="n">_seed</span> <span class="o">=</span> <span class="n">value</span>

<span class="k">def</span> <span class="nf">random</span><span class="p">():</span>
    <span class="k">global</span> <span class="n">_seed</span>
    <span class="n">_seed</span><span class="p">,</span> <span class="n">carry</span> <span class="o">=</span> <span class="nb">divmod</span><span class="p">(</span><span class="n">_seed</span><span class="p">,</span> <span class="mi">2</span><span class="p">)</span>
    <span class="k">if</span> <span class="n">carry</span><span class="p">:</span>
        <span class="n">_seed</span> <span class="o">^=</span> <span class="mh">0xb8</span>
    <span class="k">return</span> <span class="n">_seed</span>
</pre>

There are several problems with this approach.

First, it is impossible to ever instantiate a second copy of this random number generator. If two threads each wanted their own generator to avoid needing to protect it with locks, then they would be out of luck.

(Okay, not really; this is Python. Think of the possibilities! You could import the module, save a reference to it, remove it from the `sys.modules` dictionary, and then import it again to get a second copy. Or you could manually instantiate a second module object and copy all three names across. By “out of luck” I refer only to sane Pythonic approaches.)

Second, it is more difficult to decouple your random number generator tests from each other if the generator’s state is global. Instead of each test getting to create and exercise a separate isolated instance, the tests will all have to share the single generator and correctly reset its state before the next test runs.

Third, this approach abandons encapsulation. This will sound like a fuzzier complaint than the previous two, but it can detract from readability (“readability counts”) for a small tight system of two functions and a `seed` to be splayed out as three separate and not obviously related names in a module which might contain dozens of other objects.

To solve the above problems, we indent the two functions and wrap them up as methods of a new Python class. The two methods and their state can then be happily bundled together:

<pre class="cd"><span></span><span class="kn">from</span> <span class="nn">datetime</span> <span class="kn">import</span> <span class="n">datetime</span>

<span class="k">class</span> <span class="nc">Random8</span><span class="p">(</span><span class="nb">object</span><span class="p">):</span>
    <span class="k">def</span> <span class="fm">__init__</span><span class="p">(</span><span class="bp">self</span><span class="p">):</span>
        <span class="bp">self</span><span class="o">.</span><span class="n">set_seed</span><span class="p">(</span><span class="n">datetime</span><span class="o">.</span><span class="n">now</span><span class="p">()</span><span class="o">.</span><span class="n">microsecond</span> <span class="o">%</span> <span class="mi">255</span> <span class="o">+</span> <span class="mi">1</span><span class="p">)</span>

    <span class="k">def</span> <span class="nf">set_seed</span><span class="p">(</span><span class="bp">self</span><span class="p">,</span> <span class="n">value</span><span class="p">):</span>
        <span class="bp">self</span><span class="o">.</span><span class="n">seed</span> <span class="o">=</span> <span class="n">value</span>

    <span class="k">def</span> <span class="nf">random</span><span class="p">(</span><span class="bp">self</span><span class="p">):</span>
        <span class="bp">self</span><span class="o">.</span><span class="n">seed</span><span class="p">,</span> <span class="n">carry</span> <span class="o">=</span> <span class="nb">divmod</span><span class="p">(</span><span class="bp">self</span><span class="o">.</span><span class="n">seed</span><span class="p">,</span> <span class="mi">2</span><span class="p">)</span>
        <span class="k">if</span> <span class="n">carry</span><span class="p">:</span>
            <span class="bp">self</span><span class="o">.</span><span class="n">seed</span> <span class="o">^=</span> <span class="mh">0xb8</span>
        <span class="k">return</span> <span class="bp">self</span><span class="o">.</span><span class="n">seed</span>
</pre>

This imposes an extra step on the caller: the object will have to be instantiated before the two methods can be called. Is there a way to let the caller skip that step?

The software engineer would do well to stop for a moment and to think quite seriously about the fact that in the vast majority of cases simply defining a class like this is enough. It will take only a single statement for your user to create an instance of your class. If their own application’s architecture is too deeply compromised for them to easily pass the instance everywhere it’s needed, they can always store the instance as a module global in one of their own modules for the use of the rest of their code.

There are several reasons that that the Prebound Methods pattern is a particularly good fit for a random number generator:

1. Instantiating a random number generator requires a system call — in our case, asking for the date; for the Python `random` module, fetching bytes from the system entropy pool. If every module needing a random number had to instantiate its own `Random` object, then this cost would be incurred repeatedly.


2. Pseudo-random number generators are an interesting case of a resource whose behavior can be even more desirable when shared. If you are the lone caller to an instance, you see its completely predictable repeating sequence of values. If instead you are sharing that instance with other code, the generator will appear to skip ahead in its sequence unpredictably each time other callers have themselves called the generator.


3. Since most users of `random`, including several modules within the Standard Library, import it specifically to use its module-level calls, it is rare for the pre-built `Random` instance that powers them to languish unused.

If the costs and benefits strike a similar balance for a module of your own you are designing, then the Prebound Method pattern can let you deliver remarkable convenience.

## The pattern
To offer your users a slate of Prebound Methods:

+ Instantiate your class at the top level of your module.


+ Consider assigning it a private name prefixed with an underscore `_` that does not invite users to meddle with the object directly.


+ Finally, assign a bound copy of each of the object’s methods to the module’s global namespace.


For the random number generator that we used as an illustration above, the entire module might look like this:

<pre class="cd"><span></span><span class="kn">from</span> <span class="nn">datetime</span> <span class="kn">import</span> <span class="n">datetime</span>

<span class="k">class</span> <span class="nc">Random8</span><span class="p">(</span><span class="nb">object</span><span class="p">):</span>
    <span class="k">def</span> <span class="fm">__init__</span><span class="p">(</span><span class="bp">self</span><span class="p">):</span>
        <span class="bp">self</span><span class="o">.</span><span class="n">set_seed</span><span class="p">(</span><span class="n">datetime</span><span class="o">.</span><span class="n">now</span><span class="p">()</span><span class="o">.</span><span class="n">microsecond</span> <span class="o">%</span> <span class="mi">255</span> <span class="o">+</span> <span class="mi">1</span><span class="p">)</span>

    <span class="k">def</span> <span class="nf">set_seed</span><span class="p">(</span><span class="bp">self</span><span class="p">,</span> <span class="n">value</span><span class="p">):</span>
        <span class="bp">self</span><span class="o">.</span><span class="n">seed</span> <span class="o">=</span> <span class="n">value</span>

    <span class="k">def</span> <span class="nf">random</span><span class="p">(</span><span class="bp">self</span><span class="p">):</span>
        <span class="bp">self</span><span class="o">.</span><span class="n">seed</span><span class="p">,</span> <span class="n">carry</span> <span class="o">=</span> <span class="nb">divmod</span><span class="p">(</span><span class="bp">self</span><span class="o">.</span><span class="n">seed</span><span class="p">,</span> <span class="mi">2</span><span class="p">)</span>
        <span class="k">if</span> <span class="n">carry</span><span class="p">:</span>
            <span class="bp">self</span><span class="o">.</span><span class="n">seed</span> <span class="o">^=</span> <span class="mh">0xb8</span>
        <span class="k">return</span> <span class="bp">self</span><span class="o">.</span><span class="n">seed</span>

<span class="n">_instance</span> <span class="o">=</span> <span class="n">Random8</span><span class="p">()</span>

<span class="n">random</span> <span class="o">=</span> <span class="n">_instance</span><span class="o">.</span><span class="n">random</span>
<span class="n">set_seed</span> <span class="o">=</span> <span class="n">_instance</span><span class="o">.</span><span class="n">set_seed</span>
</pre>

Users will now be able to invoke each method as though it were a stand-alone function. But the methods will secretly share state thanks to the common instance that they are bound to, without requiring the user to manage or pass that state explicitly.

When exercising this pattern, please be responsible about the dangers of instantiating an object at import time. This pattern is usually not appropriate for a class whose constructor creates files, reads a database configuration, opens sockets, or that in general will inflict side effects on the program importing them. In that case, you will do better to either avoid the Prebound Methods pattern entirely, or else to defer any actual side effects until one of the methods is called. You could choose a middle ground of providing a `setup()` method and requiring application programmers to invoke it before they can expect any of the other routines to work.

But for lightweight objects that can be instantiated without substantial delay or complication, the Prebound Methods pattern is an elegant way to make the stateful behavior of a class instance available up at a module’s global level.

Examples of the pattern are strewn merrily across the Standard Library even beyond the `random` module we used as our example. The `calendar.py` module uses it:

<pre class="cd"><span></span><span class="n">c</span> <span class="o">=</span> <span class="n">TextCalendar</span><span class="p">()</span>

<span class="o">...</span>

<span class="n">monthcalendar</span> <span class="o">=</span> <span class="n">c</span><span class="o">.</span><span class="n">monthdayscalendar</span>
<span class="n">prweek</span> <span class="o">=</span> <span class="n">c</span><span class="o">.</span><span class="n">prweek</span>
<span class="n">week</span> <span class="o">=</span> <span class="n">c</span><span class="o">.</span><span class="n">formatweek</span>
<span class="n">weekheader</span> <span class="o">=</span> <span class="n">c</span><span class="o">.</span><span class="n">formatweekheader</span>
<span class="n">prmonth</span> <span class="o">=</span> <span class="n">c</span><span class="o">.</span><span class="n">prmonth</span>
<span class="n">month</span> <span class="o">=</span> <span class="n">c</span><span class="o">.</span><span class="n">formatmonth</span>
<span class="n">calendar</span> <span class="o">=</span> <span class="n">c</span><span class="o">.</span><span class="n">formatyear</span>
<span class="n">prcal</span> <span class="o">=</span> <span class="n">c</span><span class="o">.</span><span class="n">pryear</span>
</pre>

As does the venerable old `reprlib.py`, to offer a single `repr()` routine:

<pre class="cd"><span></span><span class="n">aRepr</span> <span class="o">=</span> <span class="n">Repr</span><span class="p">()</span>
<span class="nb">repr</span> <span class="o">=</span> <span class="n">aRepr</span><span class="o">.</span><span class="n">repr</span>
</pre>

As do several other modules — you will find the Prebound Methods pattern used with each of these Standard Library objects:

+ `distutils.log._global_log`


+ `multiprocessing.forkserver._forkserver`


+ `multiprocessing.semaphore_tracker._semaphore_tracker`


+ `secrets._sysrand`

One final hint: it is almost always better to assign methods to global names explicitly. Even if there are a dozen methods, I recommend going ahead and writing a quick stack of a dozen assignment statements aligned at the left-hand column of your module that make visible the whole list of global names you are defining. The fact that Python is a dynamic language might tempt you to automate the series of assignments instead, using attribute introspection and a `for` loop. I advise against it. Python programmers believe that “Explicit is better than implicit” — and materializing the stack of names as real code will better support human readers, language servers, and even venerable old `grep`.